In [1]:
#########################################  Setup the Environment Variables  #############################
import os
import re 
import sys
spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
if not os.path.isdir(spark_home):
    raise ValueError('SPARK_HOME environment variable is not a directory')
if not os.path.isdir(os.path.join(spark_home, 'python')):
    raise ValueError('SPARK_HOME directory does not contain python')
sys.path.insert(0, os.path.join(spark_home, 'python'))
pylib_list = (item for item in os.listdir(os.path.join(spark_home, 'python/lib/'))
              if re.match(r'py4j-\d+(\.\d+)+-src\.zip\Z', item)
              )
try:
    py4j_file = max(pylib_list)
    py4j = os.path.join(spark_home, os.path.join('python/lib', py4j_file))
except ValueError:
    raise ValueError(
        'Could not find py4j'
    )
sys.path.insert(0, py4j)

spark_release_file = spark_home + "/RELEASE"
if os.path.exists(spark_release_file) and "Spark" in  open(spark_release_file).read():
    pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", " --master yarn --deploy-mode client\
                                         --num-executors 10 --executor-memory 64g  --executor-cores 8 --driver-memory 140g --conf spark.dynamicAllocation.maxExecutors=2000 --jars /usr/hdp/current/hive_warehouse_connector/hive-warehouse-connector-assembly-1.0.0.3.1.4.39-6.jar  --py-files /usr/hdp/current/hive_warehouse_connector/pyspark_hwc-1.0.0.3.1.4.39-6.zip --conf spark.hadoop.metastore.catalog.default=hive"
                                        )
    if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
    os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
    print(pyspark_submit_args)
with open(os.path.join(spark_home, 'python/pyspark/shell.py')) as f:
    code = compile(f.read(), os.path.join(spark_home, 'python/pyspark/shell.py'), 'exec')
    exec(code)

"--master" "yarn" "--deploy-mode" "client" "--name" "PySparkShell" "--executor-memory" "64g" "--executor-cores" "8" "pyspark-shell"
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2.3.1.5.22-6
      /_/

Using Python version 3.6.8 (default, Dec 30 2018 01:22:34)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql.functions import udf, col, lit, array
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, FloatType

from pyspark_llap.sql.session import HiveWarehouseSession
from pyspark.sql import SparkSession
hive = HiveWarehouseSession.session(spark).build()


import time
import pandas as pd
import numpy as np
import datetime
import math
import scipy
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
import subprocess
import sys
import pyspark.sql as SQL
from pyspark.sql.functions import concat_ws
from pyspark.sql.functions import avg
from pyspark import SparkConf
from pyspark.sql import HiveContext
from pyspark import SparkContext
from pyspark.sql.functions import col,row_number
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import Row
from pyspark.sql.functions import to_timestamp,unix_timestamp,from_unixtime
from pyspark.sql.types import DoubleType, FloatType, BooleanType, LongType, IntegerType, TimestampType, StringType, StructType, StructField, ArrayType

sqlContext.setConf("spark.sql.shuffle.partitions", "2500")

ModuleNotFoundError: No module named 'pyspark_llap'

In [3]:
from pyspark_llap.sql.session import HiveWarehouseSession
from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("LLAP_Test").enableHiveSupport().config("spark.sql.hive.llap", "true").getOrCreate()
spark = SparkSession.builder.appName("Hive_ACID_warehouseconnector").enableHiveSupport().getOrCreate()
hive = HiveWarehouseSession.session(spark).build()

In [15]:
s = 'AirAmb_Te_ActlFilt Outside_Air_Temp_Stat HvacEvap_Te_Actl HvacEvap_Te_Rq AirCondFluidHi_P_Actl CoolantFanStepAct VehLong2_A_Actl HvacAirCond_B_Rq Veh_V_ActlEng ApedPos_Pc_ActlArb EngAout_N_Actl HvacBlwrFront_D_Stat BattULo_U_Actl GPS_Latitude_Degrees GPS_Longitude_Degrees GearLvrPos_D_Actl Ignition_Status DriverWindowPosition RearPassWindowPos PassWindowPosition BrkTot_Tq_Actl'
tuple(s.split(' '))


('AirAmb_Te_ActlFilt',
 'Outside_Air_Temp_Stat',
 'HvacEvap_Te_Actl',
 'HvacEvap_Te_Rq',
 'AirCondFluidHi_P_Actl',
 'CoolantFanStepAct',
 'VehLong2_A_Actl',
 'HvacAirCond_B_Rq',
 'Veh_V_ActlEng',
 'ApedPos_Pc_ActlArb',
 'EngAout_N_Actl',
 'HvacBlwrFront_D_Stat',
 'BattULo_U_Actl',
 'GPS_Latitude_Degrees',
 'GPS_Longitude_Degrees',
 'GearLvrPos_D_Actl',
 'Ignition_Status',
 'DriverWindowPosition',
 'RearPassWindowPos',
 'PassWindowPosition',
 'BrkTot_Tq_Actl')

In [4]:
df = hive.executeQuery("SELECT cvdcus_dcd_sig_n_x_3, cvdcus_dcd_sig_val_str_x_3 \
FROM cvdp.NCVDCUS_BDD_DCD_CHILD_SEC_F150_MY2018_VW WHERE cvdcus_vin_d_3 == '1FTEW1C54JKF00640' \
AND cvdcus_dcd_sig_n_x_3 IN ('HvacEvap_Te_Actl', 'HvacEvap_Te_Rq')")

In [13]:
signal = spark.sql("SELECT cvdcus_dcd_sig_n_x_3, cvdcus_dcd_sig_val_str_x_3 \
FROM cvdp.NCVDCUS_BDD_DCD_CHILD_SEC_F150_MY2018_VW WHERE cvdcus_vin_d_3 == '1FTEW1C54JKF00640' \
AND cvdcus_dcd_sig_n_x_3 IN ('HvacEvap_Te_Actl', 'HvacEvap_Te_Rq')")               

Py4JJavaError: An error occurred while calling o84.sql.
: java.lang.OutOfMemoryError: GC overhead limit exceeded
	at java.util.Arrays.copyOf(Arrays.java:3332)
	at java.lang.AbstractStringBuilder.ensureCapacityInternal(AbstractStringBuilder.java:124)
	at java.lang.AbstractStringBuilder.append(AbstractStringBuilder.java:448)
	at java.lang.StringBuffer.append(StringBuffer.java:270)
	at java.net.URI.appendSchemeSpecificPart(URI.java:1911)
	at java.net.URI.toString(URI.java:1941)
	at java.net.URI.<init>(URI.java:742)
	at org.apache.hadoop.fs.Path.initialize(Path.java:256)
	at org.apache.hadoop.fs.Path.<init>(Path.java:217)
	at org.apache.spark.sql.execution.datasources.InMemoryFileIndex$$anonfun$org$apache$spark$sql$execution$datasources$InMemoryFileIndex$$bulkListLeafFiles$3$$anonfun$7.apply(InMemoryFileIndex.scala:251)
	at org.apache.spark.sql.execution.datasources.InMemoryFileIndex$$anonfun$org$apache$spark$sql$execution$datasources$InMemoryFileIndex$$bulkListLeafFiles$3$$anonfun$7.apply

In [13]:
df1 = spark.sql('select * from cvdp.NCVDCUS_BDD_DCD_CHILD_SEC_F150_MY2018_VW limit 10')
df1.registerTempTable("connections")
a = df1.toPandas()

Py4JJavaError: An error occurred while calling o84.sql.
: java.lang.OutOfMemoryError: Java heap space


In [10]:
signal.columns

['vin',
 'epoch_new',
 'datetime',
 'AIRCONDFLUIDHI_P_ACTL',
 'APEDPOS_PC_ACTLARB',
 'CABNAMB_TE_ACTL',
 'HVACAIRFULLOUT_B_RQ',
 'HVACCMPRLIM_D_STAT',
 'HVACEVAP_TE_ACTL',
 'HVACEVAP_TE_RQ',
 'HVACREC_PC_EST',
 'ODOCOUNT',
 'ODOMETERMASTERVALUE',
 'ODOMETERMASTERVALUE_UB',
 'ODOTRIPVERIFY_L_ACTL',
 'OUTSIDE_AIR_TEMP_STAT',
 'OUTSIDE_AIR_TEMP_STAT_UB',
 'VEH_V_ACTLENG',
 'min_odo',
 'label']

# Approach 2

In [1]:
import time
import datetime
import math
import subprocess
import sys

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, countDistinct
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import Row
from pyspark.sql.types import DoubleType, FloatType, BooleanType, LongType, \
IntegerType, TimestampType, StringType, StructType, StructField, ArrayType

In [3]:
CDSID = 'achopra8'
spark_partitions = 6000
spark_big_partitions = 40000
modem_table_name = "cvdp.ncvdc62_tcuftcp_msg_sec_hte"
modem_file_path = "hdfs://hdp2cluster/project/CVDP/CVDPPRD/Warehouse/Secure/TCUFTCP/ncvdc62_tcuftcp_msg_sec_hte"
modem_data_access_method = "table"
SAVE_table_name = CDSID + ".save_na_17build_ext"
WERS_des_table_name = "cvdp.ncvdpaj_ssava03_wrsftr_hte"
output_folder = "/project/dz/collab/ccbdd"
region_code = "AP"
all_data_days = 60
dtc_data_days = 30

In [4]:
spark = SparkSession.builder \
.appName("KOL_SPC_PySpark") \
.config("spark.sql.orc.enabled", "true") \
.config("spark.sql.orc.filterPushdown", "true") \
.config("spark.sql.orc.char.enabled", "true") \
.config("spark.sql.shuffle.partitions", spark_partitions) \
.config("spark.sql.adaptive.enabled ", "true") \
.config("spark.sql.cbo.enabled", "true") \
.config("spark.sql.autoBroadcastJoinThreshold", "20971520") \
.config("spark.sql.orc.impl","native") \
.enableHiveSupport() \
.getOrCreate()

In [5]:
df = spark.sql("SELECT cvdcus_vin_d_3, cvdcus_mdl_n_3, cvdcus_mdl_yr_r_3, cvdcus_timestamp_s_3, cvdcus_dcd_sig_n_x_3, cvdcus_dcd_sig_val_str_x_3 \
FROM cvdp.ncvdcus_bdd_dcd_child_sec_f150_my2019_to_my2020_vw \
WHERE cvdcus_vin_d_3 IN \
('1FTEW1C59KFB36449', '1FTEW1C42KKC86661', '1FTEW1E49KFA29535', '1FTFW1E15KFA81450', '1FTEW1E50KKD26192', '1FTEW1E42KFA87003', '1FTEW1EP0KFA72185', '1FTEW1E45KFA14370', '1FTFX1E40KKC34508', '1FTFW1E56KFA49349', '1FTEW1E54KFA50916', '1FTEW1E4XKFA12632', '1FTEW1E46KKC51991', '1FTEW1E56KKC44273', '1FTEW1E49KFA55519', '1FTEW1EG4KFA57544', '1FTEW1E49KKC50608', '1FTFX1E59KKC27128', '1FTEW1E53KFB26139', '1FTFW1E1XKFA82819', '1FTFW1E48KFA49988', '1FTEW1E46KFA53937', '1FTFX1E58KKC74327', '1FTEW1EP1KFA04560', '1FTEW1E45KFA42461', '1FTEW1C53KFB01079', '1FTFX1E52KFA86055', '1FTFW1E17KFA36784', '1FTEW1E48KFA09227', '1FTFW1E41KKC49294', '1FTEW1E46KKC10745') \
AND cvdcus_dcd_sig_n_x_3 IN \
('AirAmb_Te_ActlFilt', 'Outside_Air_Temp_Stat', 'HvacEvap_Te_Actl', 'HvacEvap_Te_Rq', 'AirCondFluidHi_P_Actl', 'CoolantFanStepAct', 'VehLong2_A_Actl', 'HvacAirCond_B_Rq', 'Veh_V_ActlEng', 'ApedPos_Pc_ActlArb', 'EngAout_N_Actl', 'HvacBlwrFront_D_Stat', 'BattULo_U_Actl', 'GPS_Latitude_Degrees', 'GPS_Longitude_Degrees', 'GearLvrPos_D_Actl', 'Ignition_Status', 'DriverWindowPosition', 'RearPassWindowPos', 'PassWindowPosition', 'BrkTot_Tq_Actl') \
AND MONTH(cvdcus_timestamp_s_3) BETWEEN 5 and 8")

df


Py4JJavaError: An error occurred while calling o82.sql.
: java.lang.OutOfMemoryError: GC overhead limit exceeded
	at java.util.Arrays.copyOfRange(Arrays.java:3664)
	at java.lang.StringBuffer.toString(StringBuffer.java:669)
	at java.net.URI.toString(URI.java:1945)
	at java.net.URI.<init>(URI.java:742)
	at org.apache.hadoop.fs.Path.initialize(Path.java:256)
	at org.apache.hadoop.fs.Path.<init>(Path.java:217)
	at org.apache.spark.sql.execution.datasources.InMemoryFileIndex$$anonfun$org$apache$spark$sql$execution$datasources$InMemoryFileIndex$$bulkListLeafFiles$3$$anonfun$7.apply(InMemoryFileIndex.scala:251)
	at org.apache.spark.sql.execution.datasources.InMemoryFileIndex$$anonfun$org$apache$spark$sql$execution$datasources$InMemoryFileIndex$$bulkListLeafFiles$3$$anonfun$7.apply(InMemoryFileIndex.scala:244)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collec

In [ ]:
df.show(5)